In [2]:
!pip install efficientnet
!pip install tensorflow
!pip install keras

In [1]:
import efficientnet.tfkeras as efn  # Convolutional Neural Network architecture
import IPython.display as ipd
# import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from efficientnet.keras import preprocess_input
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils import class_weight
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
IM_SIZE = (224, 224) # changed this line
BIRDS =    ['0Asian', '1Commo', '2White', '3White', '4Red-v', '5Commo']
   
DATA_PATH = "Final_data/"
BATCH_SIZE = 20

In [3]:
train_datagen = ImageDataGenerator(  # Generate batches of tensor image data with real-time data augmentation.
    preprocessing_function=preprocess_input,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.1,
    fill_mode="nearest",
)
train_batches = train_datagen.flow_from_directory(
    DATA_PATH + "train",
    classes=BIRDS,
    target_size=IM_SIZE,
    color_mode = "rgb",
    class_mode="categorical",
    shuffle=True,
    batch_size=BATCH_SIZE,
)

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
valid_batches = valid_datagen.flow_from_directory(
    DATA_PATH + "val",
    classes=BIRDS,
    target_size=IM_SIZE,
    color_mode = "rgb",
    class_mode="categorical",
    shuffle=False,
    batch_size=BATCH_SIZE,
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_batches = test_datagen.flow_from_directory(
    DATA_PATH + "test",
    classes=BIRDS,
    target_size=IM_SIZE,
    class_mode="categorical",
    shuffle=False,
    batch_size=BATCH_SIZE,
)

Found 3339 images belonging to 6 classes.
Found 319 images belonging to 6 classes.
Found 371 images belonging to 6 classes.


In [4]:
# Estimate class weights for unbalanced dataset
class_weights = class_weight.compute_class_weight(
   class_weight = "balanced", classes = np.unique(train_batches.classes),y= train_batches.classes
)
class_weights = {i:w for i,w in enumerate(class_weights)}


ReduceLR = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=4, min_lr=1e-4)

In [9]:
# Define CNN's architecture - CNN is a forward-feed network which automatically detects the important features  ( supervised type of Deep learning)
net = efn.EfficientNetB7(
    include_top=False, weights="imagenet", input_tensor=None, input_shape=(224,224,3)
)
x = net.output
x = Flatten()(x)
#  Flatten -> Lots in a straight array
x = Dense(512,activation = "relu")(x)
x = Dense(256,activation = "relu")(x)
x = Dropout(0.18)(x)
x = Dense(128,activation = "relu")(x)
x = Dense(32,activation = "relu")(x)
x = Dropout(0.08)(x)
x = Dense(16,activation = "relu")(x)


output_layer = Dense(len(BIRDS), activation="softmax", name="softmax")(x)
net_final = Model(inputs=net.input, outputs=output_layer)
net_final.compile(
    optimizer=Adam(learning_rate=0.0003), loss="categorical_crossentropy", metrics=["accuracy"]
)

# print(net_final.summary())

In [10]:
# Train the model
net_final.fit(
    train_batches,
    validation_data=valid_batches,
    epochs=40,
    verbose=1,
    steps_per_epoch=100,
    class_weight=class_weights,
     callbacks=[ ReduceLR],
    
)

Epoch 1/40
100/100 [==============================] - 98s 674ms/step - loss: 1.9263 - accuracy: 0.2360 - val_loss: 1.7764 - val_accuracy: 0.2320 - lr: 1.0000e-04
Epoch 2/40
 34/100 [=========>....................] - ETA: 39s - loss: 1.8021 - accuracy: 0.2706

2022-07-19 19:07:43.680930: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.59GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-07-19 19:07:43.680993: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.59GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-07-19 19:07:43.696388: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.54GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-07-19 19:07:43.696442: W tensorflow/core/common_runtime/bfc_allocato

100/100 [==============================] - 65s 649ms/step - loss: 1.7378 - accuracy: 0.2983 - val_loss: 1.6101 - val_accuracy: 0.3793 - lr: 1.0000e-04
Epoch 3/40
100/100 [==============================] - 63s 632ms/step - loss: 1.6185 - accuracy: 0.3363 - val_loss: 1.4458 - val_accuracy: 0.3981 - lr: 1.0000e-04
Epoch 4/40
100/100 [==============================] - 64s 640ms/step - loss: 1.4328 - accuracy: 0.4490 - val_loss: 1.4337 - val_accuracy: 0.4545 - lr: 1.0000e-04
Epoch 5/40
100/100 [==============================] - 62s 620ms/step - loss: 1.4279 - accuracy: 0.4390 - val_loss: 1.3315 - val_accuracy: 0.4734 - lr: 1.0000e-04
Epoch 6/40
100/100 [==============================] - 61s 613ms/step - loss: 1.2818 - accuracy: 0.5230 - val_loss: 1.0546 - val_accuracy: 0.5987 - lr: 1.0000e-04
Epoch 7/40
100/100 [==============================] - 62s 617ms/step - loss: 1.1632 - accuracy: 0.5700 - val_loss: 1.0954 - val_accuracy: 0.5987 - lr: 1.0000e-04
Epoch 8/40
100/100 [===================

In [ ]:
net_final.evaluate(test_batches)

38/38 [==============================] - 5s 132ms/step - loss: 0.6885 - accuracy: 0.8113


2022-07-19 19:54:54.911595: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.54GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-07-19 19:54:54.911652: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.54GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


[0.6884739995002747, 0.8113207817077637]